# Applatissement de la base de donnée LEGI

## Documentation

La base de donnée LEGI est mise à disposition par la DILA sous une forme complexe à 16 niveaux d'arborescence, alors que seuls 3 niveaux sont nécessaires.

Pour le système d'arborescence original, se reporter au document `Documentation_mise_à_dispo_XML_basesLegifrance_2014_11_17_002.doc`

Le premier niveau du nouveau système d'arborescence "plat" se compose de 6 dossiers :
* code_en_vigueur_LEGI (435387 fichiers, 75 cid)
* code_non_vigueur_LEGI (14692 fichiers, 31 cid)
* TNC_en_vigueur_JORF (868528 fichiers, 76944 cid)
* TNC_en_vigueur_LEGI (15807 fichiers, 1671 cid)
* TNC_non_vigueur_JORF (264268 fichiers, 17434 cid)
* TNC_non_vigueur_LEGI (25637 fichiers, 1771 cid)

Chacun de ces dossier est organisé ainsi :
* dossiers LEGITEXTxxxxxxxxxxxx (pour les dossiers \*LEGI) ou JORFTEXTxxxxxxxxxxxx (pour les dossiers \*JORF)
 * LEGIARTIxxxxxxxxxxxx.xml (1247771 fichiers)
 * LEGISCTAxxxxxxxxxxxx.xml (177124 fichiers)
 * struct : contient LEGITEXTxxxxxxxxxxxx.xml (99480 fichiers)
 * version : contient LEGITEXTxxxxxxxxxxxx.xml (99944 fichiers)


Le nombre total de fichiers n'est pas changé par la transformation (1624319 fichiers)




In [1]:
import os
import re
import shutil

In [2]:
racine_source = '/home/michel/legi/'
racine_cible = '/home/michel/legi_plat/'

In [5]:
def liste_dir(d):
    for f in os.listdir(d):
        print(f + '({})'.format(os.path.isfile(f)))

In [6]:
liste_dir(racine_source)

global(False)


In [7]:
liste_dir(racine_source + 'global')

code_et_TNC_non_vigueur(False)
code_et_TNC_en_vigueur(False)


In [11]:
liste_dir(racine_source + 'global/code_et_TNC_non_vigueur/TNC_non_vigueur/LEGI/TEXT')

00(False)


## Crée la nouvelle structure

In [7]:
for rep in [                  # find . -type f | wc -l
    'TNC_en_vigueur_JORF',    # 868528 (ok)
    'TNC_en_vigueur_LEGI',    # 15807 (ok)
    'code_en_vigueur_LEGI',   # 435387 (ok)
    'TNC_non_vigueur_JORF',   # 264268 (ok)
    'TNC_non_vigueur_LEGI',   # 25637 (ok)
    'code_non_vigueur_LEGI',  # 14692 (ok)
]:                            # total : 1624319
    os.mkdir(racine_cible + rep)

## Aplatit

In [3]:
class EnfantNonDesire(Exception):
    def __init__(self, chemin):
        self.chemin = chemin
        
    def __str__(self):
        return str(chemin)

In [4]:
def parcours(chemin, repertoire_cible):
    chemin_compact = os.path.join(*chemin)
    profondeur = len(chemin)
    
    enfants = os.listdir(chemin_compact)
    est_fichier_liste = [os.path.isfile(os.path.join(chemin_compact, f)) for f in enfants]
    enfants_fichier = [enfants[i] for i in range(len(enfants)) if est_fichier_liste[i]]
    enfants_dossier = [enfants[i] for i in range(len(enfants)) if not est_fichier_liste[i]]

    if profondeur == 1:
        assert len(enfants_fichier) == 0, chemin
        assert len(enfants_dossier) == 1, chemin
        assert enfants_dossier[0] == 'TEXT', chemin
        
    elif (profondeur >= 2) and (profondeur <= 6):
        assert len(enfants_fichier) == 0, chemin
        for enfant_dossier in enfants_dossier:
            assert re.match('^\d{2}$', enfant_dossier), chemin
            
    elif profondeur == 7:
        assert len(enfants_fichier) == 0, chemin
        for enfant_dossier in enfants_dossier:
            assert re.match('^(JORF|LEGI)TEXT\d{12}$', enfant_dossier), chemin
            os.mkdir(os.path.join(repertoire_cible, enfant_dossier))
            
    elif profondeur == 8:
        assert len(enfants_fichier) == 0, chemin
        assert len(enfants_dossier) <= 3, chemin
        assert set(enfants_dossier) <= set(['texte', 'article', 'section_ta']), chemin
    
    elif profondeur == 9:
        if chemin[8] == 'texte':
            assert len(enfants_fichier) == 0, chemin
            assert len(enfants_dossier) == 2, chemin
            assert set(enfants_dossier) == set(['struct', 'version']), chemin
            
            source = os.path.join(chemin_compact, 'struct')
            cible = os.path.join(repertoire_cible, chemin[7], 'struct')
            assert not os.path.exists(cible), chemin
            shutil.copytree(source, cible)    

            source = os.path.join(chemin_compact, 'version')
            cible = os.path.join(repertoire_cible, chemin[7], 'version')
            assert not os.path.exists(cible), chemin
            shutil.copytree(source, cible)
            
            enfants_dossier = []

        elif chemin[8] in ['article', 'section_ta']:
            assert len(enfants_fichier) == 0, chemin
            assert len(enfants_dossier) == 1, chemin
            assert enfants_dossier[0] == 'LEGI', chemin
            
        else:
            raise EnfantNonDesire(chemin)
            
    elif profondeur == 10:
        assert chemin[9] == 'LEGI', chemin
        assert len(enfants_fichier) == 0, chemin
        assert len(enfants_dossier) == 1, chemin
        if chemin[8] == 'article':
            assert enfants_dossier[0] == 'ARTI', chemin
        elif chemin[8] == 'section_ta':
            assert enfants_dossier[0] == 'SCTA', chemin
        else:
            raise EnfantNonDesire(chemin)                
            
    elif (profondeur >= 11) and (profondeur <= 15):
        assert len(enfants_fichier) == 0, chemin
        for enfant_dossier in enfants_dossier:
            assert re.match('^\d{2}$', enfant_dossier), chemin
    
    elif profondeur == 16:
        assert len(enfants_dossier) == 0, chemin
        for enfant_fichier in enfants_fichier:
            if chemin[8] == 'article':
                assert re.match('^LEGIARTI\d{12}\.xml$', enfant_fichier), chemin
            elif chemin[8] == 'section_ta':
                assert re.match('^LEGISCTA\d{12}\.xml$', enfant_fichier), chemin
            else:
                raise EnfantNonDesire(chemin)
            
            source = os.path.join(chemin_compact, enfant_fichier)
            cible = os.path.join(repertoire_cible, chemin[7], enfant_fichier)
            assert not os.path.exists(cible), chemin
            shutil.copy(source, cible)    
    
    for enfant_dossier in enfants_dossier:
        parcours(chemin + [enfant_dossier], repertoire_cible)


In [11]:
repertoire_source = os.path.join(racine_source, 'global/code_et_TNC_en_vigueur/TNC_en_vigueur/JORF')
repertoire_cible = os.path.join(racine_cible, 'TNC_en_vigueur_JORF')

parcours([repertoire_source], repertoire_cible)

In [9]:
repertoire_source = os.path.join(racine_source, 'global/code_et_TNC_en_vigueur/TNC_en_vigueur/LEGI')
repertoire_cible = os.path.join(racine_cible, 'TNC_en_vigueur_LEGI')

parcours([repertoire_source], repertoire_cible)

In [8]:
repertoire_source = os.path.join(racine_source, 'global/code_et_TNC_en_vigueur/code_en_vigueur/LEGI')
repertoire_cible = os.path.join(racine_cible, 'code_en_vigueur_LEGI')

parcours([repertoire_source], repertoire_cible)

In [6]:
repertoire_source = os.path.join(racine_source, 'global/code_et_TNC_non_vigueur/TNC_non_vigueur/JORF')
repertoire_cible = os.path.join(racine_cible, 'TNC_non_vigueur_JORF')

parcours([repertoire_source], repertoire_cible)

In [12]:
repertoire_source = os.path.join(racine_source, 'global/code_et_TNC_non_vigueur/TNC_non_vigueur/LEGI')
repertoire_cible = os.path.join(racine_cible, 'TNC_non_vigueur_LEGI')

parcours([repertoire_source], repertoire_cible)

In [5]:
repertoire_source = os.path.join(racine_source, 'global/code_et_TNC_non_vigueur/code_non_vigueur/LEGI')
repertoire_cible = os.path.join(racine_cible, 'code_non_vigueur_LEGI')

parcours([repertoire_source], repertoire_cible)